# Chapter 7: Ensemble Learning and Random Forests

Notebook ini merupakan hasil reproduksi dan penjelasan teori dari **Bab 7 - Ensemble Learning and Random Forests** dari buku *Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow (2nd Edition)* oleh Aurélien Géron.

📌 Bab ini menjelaskan konsep model ensemble, termasuk voting, bagging, Random Forests, boosting, dan teknik lain yang digunakan untuk meningkatkan performa prediksi.

---


## Ringkasan Teori Bab 7: Ensemble Learning and Random Forests

### 1. Apa Itu Ensemble Learning?

**Ensemble Learning** adalah teknik menggabungkan prediksi dari beberapa model untuk meningkatkan akurasi.  
Alih-alih mengandalkan satu model yang kuat, kita menggunakan banyak model lemah (weak learners) secara bersamaan.

**Jenis umum ensemble:**
- **Bagging (Bootstrap Aggregating)**: melatih model yang sama pada subset acak data
- **Boosting**: melatih model secara berurutan, dengan fokus pada kesalahan sebelumnya
- **Voting**: menggabungkan prediksi beberapa model (klasifikasi)
- **Stacking**: menggunakan model lain (meta-learner) untuk menggabungkan output model dasar

---

### 2. Voting Classifier

- **Hard Voting**: mayoritas suara dari masing-masing model
- **Soft Voting**: rata-rata probabilitas prediksi, lalu pilih probabilitas tertinggi

```python
from sklearn.ensemble import VotingClassifier
```

---

### 3. Bagging dan Pasting

- **Bagging**: sampling *dengan* pengembalian (bootstrap sampling)
- **Pasting**: sampling *tanpa* pengembalian

Keduanya dapat digunakan untuk mengurangi varians model.

Scikit-learn:
```python
from sklearn.ensemble import BaggingClassifier
```

---

### 4. Random Forest

Random Forest = banyak Decision Tree yang dilatih secara bagging, ditambah pemilihan fitur acak di setiap split.

```python
from sklearn.ensemble import RandomForestClassifier
```

Kelebihan:
- Kuat terhadap overfitting
- Bisa menghitung feature importance
- Cepat dan paralel

---

### 5. Boosting

Boosting melatih model secara berurutan, di mana setiap model mencoba memperbaiki kesalahan dari model sebelumnya.

Dua pendekatan populer:
- **AdaBoost**: penyesuaian bobot pada setiap sampel
- **Gradient Boosting**: model baru dilatih untuk memprediksi *residual error* dari model sebelumnya

```python
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
```

---

### 6. Feature Importance dari Random Forest

Kita bisa mengukur pentingnya setiap fitur berdasarkan berapa banyak impurity yang berhasil dikurangi oleh fitur tersebut secara rata-rata di seluruh pohon.

---

### 7. Out-of-Bag Evaluation (OOB)

Pada Bagging, kita dapat menggunakan instance yang tidak dipilih (out-of-bag) untuk mengevaluasi model, tanpa perlu validation set eksplisit.

```python
BaggingClassifier(oob_score=True)
```

---


## Implementasi: Voting Classifier

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, random_state=42)

log_clf = LogisticRegression(max_iter=1000)
tree_clf = DecisionTreeClassifier()
svm_clf = SVC(probability=True)

voting_clf = VotingClassifier(
    estimators=[('lr', log_clf), ('dt', tree_clf), ('svc', svm_clf)],
    voting='soft'
)

voting_clf.fit(X_train, y_train)
for clf in (log_clf, tree_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

## Implementasi: Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=100, max_leaf_nodes=16, n_jobs=-1, random_state=42)
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print("Random Forest accuracy:", accuracy_score(y_test, y_pred_rf))

## Visualisasi Feature Importance

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

importances = rf_clf.feature_importances_
indices = np.argsort(importances)[::-1]
names = [iris.feature_names[i] for i in indices]

plt.figure(figsize=(8, 5))
plt.title("Feature Importance (Random Forest)")
plt.bar(range(X_train.shape[1]), importances[indices])
plt.xticks(range(X_train.shape[1]), names, rotation=45)
plt.tight_layout()
plt.show()

## Implementasi: Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb_clf = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_clf.fit(X_train, y_train)
y_pred_gb = gb_clf.predict(X_test)
print("Gradient Boosting accuracy:", accuracy_score(y_test, y_pred_gb))

## Out-of-Bag Evaluation

In [ ]:
from sklearn.ensemble import BaggingClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=100,
    bootstrap=True, oob_score=True, n_jobs=-1, random_state=42)
bag_clf.fit(X_train, y_train)
print("OOB Score:", bag_clf.oob_score_)